<a href="https://colab.research.google.com/github/AbdalazizOth/Tradtional_dict/blob/main/Qamus_Muhit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[QamusMuhit_dict](https://github.com/OpenITI/0825AH/tree/master/data/0817MajdDinFiruzabadi/0817MajdDinFiruzabadi.QamusMuhit)**

In [ ]:
import re                               # RegEx module for regular expressions
import pandas as pd                     # Python library for data manipulation
from openpyxl import load_workbook      # Python library to read/write Excel files
from openpyxl.styles import Alignment

# File paths
input_file = "/content/0817MajdDinFiruzabadi.QamusMuhit.Shamela0007283-ara1"
output_file_txt = "QamusMuhit.txt"
output_file_xlsx = "QamusMuhit.xlsx"

# Regex patterns
page_pattern = re.compile(r"PageV\d{2}P\d+")          # Page number pattern like PageV01P660

root_pattern = re.compile(r"^# •\s*(.+?)[،:]\s*(.+)") # Root pattern (starting with # •)

# Initialize variables
data = []
page = None             # Current active page
next_page = None        # Temporarily stores page found inside text until next root
root = None
text_lines = []
root_started = False    # Tracks if we are inside a root

# Process the file
with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        line = line.rstrip()
        if re.match(r"^\s*### \|", line):
            continue

        # Detect page marker
        if match := page_pattern.search(line):
            if root_started == False:

                page = match.group(0)          # Normal case: page appears before root
            else:

                next_page = match.group(0)     # New page appears inside text → belongs to next root
            continue


        # Detect new root
        if match := root_pattern.match(line):

            if root is not None:                       # Save previous root if exists
                text = " ".join(text_lines).strip()
                data.append((page, root, text))


            # Use any saved "next_page" for this new root
            if next_page is not None:
                page = next_page
                next_page = None

            # Extract new root and first line of text
            root = match.group(1).strip()
            root = re.sub(r"\bms\d{4}\b", "", root)
            first_example_part = match.group(2).strip()
            first_example_part = re.sub(r"\bms\d{4}\b|«\d+»|\|\||\.\.\.|\?\?|\=\:|=|•", "", first_example_part)
            text_lines = []

            if first_example_part:
                text_lines.append(first_example_part)

            root_started = True  # now inside a new root
            continue

        # Collect text lines under current root
        if root and line.strip():
            line=re.sub(r"\bms\d{4}\b|«\d+»|\#|\~\~||\{|\}|\|\||\.\.\.|\?\?|\s+\:|\)\:|\=\:|=|•", "",line)
            text_lines.append(line.strip())

# Save last root
if root and text_lines:
    text = " ".join(text_lines).strip()
    data.append((page, root, text))
    root_started = False



# 1️⃣ Save text file
with open(output_file_txt, "w", encoding="utf-8") as out:
    for page, root, text in data:
        out.write(f"Page: {page}\nRoot: {root}\nText: {text}\n\n")

# 2️⃣ Save Excel file
df = pd.DataFrame(data, columns=["Page", "Root", "Text"])
df.to_excel(output_file_xlsx, index=False, engine="openpyxl")

# 3️⃣ Format Excel (RTL + wrap text)
wb = load_workbook(output_file_xlsx)
ws = wb.active

# Make sheet right-to-left
ws.sheet_view.rightToLeft = True

# Apply text alignment and wrapping for all cells
align = Alignment(wrap_text=True, horizontal="right", vertical="top")
for row in ws.iter_rows():
    for cell in row:
        cell.alignment = align

# Adjust column width (fixed reasonable size)
ws.column_dimensions["A"].width = 15   # Page
ws.column_dimensions["B"].width = 20   # Root
ws.column_dimensions["C"].width = 80   # Text


wb.save(output_file_xlsx)

print(f"✅ Done! {len(data)} roots extracted.")  # to determine the number of items in data list
print(f"📄 Text file: {output_file_txt}")
print(f"📊 Excel file: {output_file_xlsx}")

✅ Done! 10263 roots extracted.
📄 Text file: QamusMuhit.txt
📊 Excel file: QamusMuhit.xlsx
